In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from google.colab import files
uploaded = files.upload()
import io

Saving charity_data.csv to charity_data.csv


In [2]:
application_df = pd.read_csv(io.StringIO(uploaded['charity_data.csv'].decode('utf-8')))

In [3]:
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
application_df = application_df.drop(["EIN", "NAME"], axis=1)
application_types_to_replace = ["T13","T12","T2", "T14","T25", "T15", "T29","T17"]
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [5]:
lower_classification = application_df['CLASSIFICATION'].value_counts().loc[lambda x : x<1000]
classifications_to_replace = lower_classification.index
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [6]:
df = pd.get_dummies(application_df)
df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8', 'APPLICATION_TYPE_T9',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-

In [7]:
y = df["IS_SUCCESSFUL"].values
X = df.drop(["IS_SUCCESSFUL"],axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4)

In [8]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
nn = tf.keras.models.Sequential()
number_input_features = len(X_train[0])


nn.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim= number_input_features))
# First hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu"))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu"))
# Tgird hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 225       
                                                                 
 dense_1 (Dense)             (None, 5)                 30        
                                                                 
 dense_2 (Dense)             (None, 5)                 30        
                                                                 
 dense_3 (Dense)             (None, 5)                 30        
                                                                 
 dense_4 (Dense)             (None, 1)                 6         
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6388 - accuracy: 0.6420
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5768 - accuracy: 0.7221
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5659 - accuracy: 0.7243
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5607 - accuracy: 0.7271
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5577 - accuracy: 0.7282
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5553 - accuracy: 0.7274
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5539 - accuracy: 0.7290
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5526 - accuracy: 0.7291
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5517 - accuracy: 0.7294
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5513 - accura

In [32]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5584 - accuracy: 0.7268 - 384ms/epoch - 1ms/step
Loss: 0.5584033131599426, Accuracy: 0.7267638444900513


In [13]:
df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [20]:
y = df["IS_SUCCESSFUL"].values
X = df.drop(["IS_SUCCESSFUL"],axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4)

In [21]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
def create_model(hp):
    number_input_features = len(X_train[0])
    nn_model = tf.keras.models.Sequential()
    activation = hp.Choice('activation',['relu'])
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=number_input_features))

    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [30]:
!pip install keras_tuner
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 49s]
val_accuracy: 0.7303789854049683

Best val_accuracy So Far: 0.7327113747596741
Total elapsed time: 00h 22m 35s


In [33]:
top_hyper = tuner.get_best_hyperparameters(1)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 26, 'num_layers': 4, 'units_0': 11, 'units_1': 11, 'units_2': 1, 'units_3': 16, 'units_4': 11, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}


In [34]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(1)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5597 - accuracy: 0.7327 - 574ms/epoch - 2ms/step
Loss: 0.5597481727600098, Accuracy: 0.7327113747596741


In [36]:
nn_model = tf.keras.models.Sequential()
number_input_features = len(X_train[0])


nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim= number_input_features))
# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu"))
# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu"))
# Tgird hidden layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu"))
# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="tanh"))


# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 5)                 225       
                                                                 
 dense_12 (Dense)            (None, 5)                 30        
                                                                 
 dense_13 (Dense)            (None, 5)                 30        
                                                                 
 dense_14 (Dense)            (None, 5)                 30        
                                                                 
 dense_15 (Dense)            (None, 1)                 6         
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [37]:
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.8902 - accuracy: 0.5480
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6196 - accuracy: 0.6728
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5967 - accuracy: 0.6996
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5835 - accuracy: 0.7110
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5724 - accuracy: 0.7192
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5672 - accuracy: 0.7220
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5647 - accuracy: 0.7229
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5622 - accuracy: 0.7242
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5623 - accuracy: 0.7259
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5613 - accura

In [40]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5627 - accuracy: 0.7255 - 378ms/epoch - 1ms/step
Loss: 0.5627152323722839, Accuracy: 0.7254810333251953


In [41]:
model_json = nn.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
nn.save_weights("AlphabetSoupCharity_Optimisation.h5")
print("Saved model to disk")

Saved model to disk
